<a href="https://colab.research.google.com/github/FeliciaSchenkelberg/Algorithms-and-Data-Structure/blob/main/Copy_of_Assignment_6_Fall2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ENGS 108 Fall 2022 Assignment 6**

*Due November 11, 2022 at 11:59PM on Github*

**Instructors:** George Cybenko

**TAs:** Chase Yakaboski and Clement Nyanhongo


---

## **Rules and Requirements**


1.   You are only allowed to use Python packages that are explicity imported in 
the assignment notebook or are standard (bultin) python libraries like random, os, sys, etc, (Standard Bultin Python libraries will have a Python.org documentation). For this assignment you may use:
  *   [numpy](https://numpy.org/doc/stable/)
  *   [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
  *   [scikit-learn](https://scikit-learn.org/stable/)
  *   [matplotlib](https://matplotlib.org/)
  *   [tensorflow](https://www.tensorflow.org/)

2.   All code must be fit into the designated code or text blocks in the assignment notebook. They are indentified by a **TODO** qualifier.

3. For analytical questions that don't require code, type your answer cleanly in Markdown. For help, see the [Google Colab Markdown Guide](https://colab.research.google.com/notebooks/markdown_guide.ipynb).

---


In [ ]:
''' Import Statements '''
import os
import random
import numpy as np
import pandas as pd
import csv
import tqdm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Creating a Trump Tweet Generator!!!
Finally it's happened. I am allowed to make my dream assignment. Trust me it's the best assignment. People tell me all the time how they wish they could have had an assignment as great as this. You will see why! But first some housekeeping... 

# Data Loading
In this assignment we will be using a repository of Donald Trump tweets scrapped from Twitter through June 2020 from [Kaggle](https://www.kaggle.com/datasets/austinreese/trump-tweets) and will use the following code blocks to download the dataset directly to your Google Drive.

## Creating a Kaggle API Token
First we will need to download an API token from Kaggle in order to download the dataset, so our first step is to create a Kaggle account if you don't already have one. (You should have done this in Assignment 4 in case this looks familiar. 
1. Create a Kaggle account by following the sign up instructions [here](https://www.kaggle.com/).
2. Log into your Kaggle account and click your account icon on the upper righthand side. 
3. Then select **Account** from the dropdown/sidebar menu.
4. Scroll down to the **API** section and select **Create New API Token**.
5. This will download a JSON file called kaggle.json to your Downloads folder on your computer.
6. Now run the following code block and when the **Browse** button appears, click it and select that kaggle.json file. 

In [ ]:
# Run this code block after creating a Kaggle API token as instructed above.
! pip install -q kaggle
from google.colab import files 

# Will ask you to upload kaggle.json file and remove any old ones.
if os.path.exists('kaggle.json'):
  os.remove('kaggle.json')
files.upload()

# Will create the appropriate directory structure
if not os.path.exists('/root/.kaggle'):
  ! mkdir ~/.kaggle 
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
# Also we are going to make a directory called result
if not os.path.exists('/content/results'):
  ! mkdir /content/result

Saving kaggle.json to kaggle.json


## Downloading the Dataset

7. Now we have downloaded our Kaggle credentials we can now download the Trump Tweets Dataset (or any other Kaggle dataset for that matter) directly into our Google Drive.

In [ ]:
! kaggle datasets download austinreese/trump-tweets
# Will check to see if the yoga postures zip file has been unzipped and will unzip the file if not.
! unzip trump-tweets.zip

 87% 6.00M/6.88M [00:00<00:00, 59.8MB/s]
100% 6.88M/6.88M [00:00<00:00, 62.0MB/s]
Archive:  trump-tweets.zip
  inflating: realdonaldtrump.csv     
  inflating: trumptweets.csv         


## Loading the Dataset using Pandas
Now let's inspect the trump tweets dataset and see what we have to work with... Brace yourselves.

In [ ]:
# Let's load in the two files that we inflated from the Kaggle download. Both realdonaldtrump.csv and trumptweets.csv are the same.
real_donald_trump_df = pd.read_csv('realdonaldtrump.csv')

In [ ]:
real_donald_trump_df.head()

,id,link,content,date,retweets,favorites,mentions,hashtags
0,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,510,917,NaN,NaN
1,1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,34,267,NaN,NaN
2,1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,13,19,NaN,NaN
3,1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,11,26,NaN,NaN
4,1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 09:07:28,1375,1945,NaN,NaN


## Problem 1: Pre-Processing the Tweets
As you may have noticed from the dataframe we just loaded, there are some special characters we need to handle. If we are making a tweet or sentence generator, we don't want to mess with special characters like commas or colons or really even captialization. So in the following section you are going to preprocess the data and strip these special characters out.

### Task 1: The Preprocess function
In the following code block complete the preprocess function that will strip or substitute out various special characters or sequences of characters that may not be ideal when training a sentence generator. We will use the built-in re python library to do a number of substitutions, and I have given you a skeleton below, see if you can complete it.

In [ ]:
import re

REGEX_SUBS = {
  r'\\': ' ',
  r'\n': ' ',
  r'&': '',
  r'RT ': '',
  r'~': '',
  r'#': '',
  r'!+': '',
  r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)': 'link',
  r'[*]': '',
  r'[@]\w+': 'user',
  r'[:|;]': '',
  r'[\\x]\W+|\d': '',
  r'[\\x]\W+|\d': '',
  r'[\\x]\w+': '',
  r'    ': ' ',
  r'   ': ' ',
  r'  ': ' ',
}

def preprocess(text):
  for key, value in REGEX_SUBS.items():
    text = re.sub(key, value, text)
    text = text.lower()
  return text

### Task 2: Preprocess the dataset.
Now that we have our preprocess function, let's preprocess all the tweets.

In [ ]:
from nltk import tokenize

sentence = []

for x in real_donald_trump_df["content"]:
  for y in tokenize.sent_tokenize(preprocess(x)):
    sentence.append(re.sub(r'[^\w\s]', '', y))

In [ ]:
cleaned_data = []

for i in sentence:
    if i not in cleaned_data:
        cleaned_data.append(i.split())

In [ ]:
cleaned_data[5]

['miss',
 'usa',
 'tara',
 'conner',
 'will',
 'not',
 'be',
 'fired',
 'ive',
 'always',
 'been',
 'a',
 'believer',
 'in',
 'second',
 'chances']

# Problem 1: N-gram Language Model
In this problem, you will be building a couple n-gram language modeling and see how well just taking pure frequency counts and building a conditional probability distrbution will work.

## Task 1: A 2-gram (Bigram) Model
Recall that our goal in building a language model is to represent the conditional probability $P(w_i | w_{i-1})$ for pairs of words $w_i$ and $w_j$.

In [ ]:
n_grams = {}

In [ ]:
for data in cleaned_data:
  words = [word for word in data if word[0].isalpha()]
  for ix in range(len(words) -1):
    try:
      n_grams[words[ix]].append(words[ix + 1])
    except KeyError as _:
      n_grams[words[ix]] = []
      n_grams[words[ix]].append(words[ix + 1])

In [ ]:
def generate_text():
  words = []
  next_word = random.choice(list(n_grams.keys()))
  words.append(next_word)

  while len(words) < random.randint(10, 15):
    next_word = random.choice(n_grams[next_word])
    words.append(next_word)

  return " ".join(words)

In [ ]:
generate_text()

'amendment for receiving an unsustainable border wall being made a great all'

# Problem 2: Using a Transformer
In this section we are going to leverage a pretrained transformer, i.e., [GPT-2](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf), built by the [Hugging Faces Team](https://huggingface.co/gpt2?text=A+long+time+ago%2C). We will also be using their tokenizers because they have been optimized for the language generation task. You should be aware that behind the scenes, their model is using [PyTorch](https://pytorch.org/), the deep learning library built by Facebook, and is quickly becoming more popular than our beloved Tensorflow. 

## Task 1: Install and load the GPT2 Model

In [ ]:
# Run the following code to install the HuggingFace's transformer python package.
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.4 MB/s 
     |████████████████████████████████| 163 kB 65.4 MB/s 
     |████████████████████████████████| 7.6 MB 51.3 MB/s 


In [ ]:
# TODO: Follow the link above and load the GPT2 model as well as the tokenizer.
# NOTE: Replace GPT2Model with GPT2LMHeadModel (this is the model for finetuning use cases.)
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

## Task 2: Formatting our Training Data
Upcoming we will be fine-tunning this GPT-2 model on our Trump Tweets, but in order to leverage some of the utility classes built by HuggingFaces, we want to take our preprocessed Trump tweets and place them in a flat text file.

In [ ]:
#TODO: Take our preprocessed trump tweets and write them to a text file
with open('trumpdata.txt', 'w') as text_file:
    for sent in cleaned_data:
      text_file.write("%s\n" % sent)

## Task 4: Fine-tuning the Model
In the following sections, we will complete a couple functions that will allow us to fine-tune the GPT-2 model to our Trump Tweets. I am going to give you a couple of these helper functions, but leave you to write parts of the training function. See the following documentation from Hugging Faces to see the attributes for the [Trainer Class](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer).

In [ ]:
#TODO: Set necessary parameters, here are some defaults.
train_file_path = "/content/trumpdata.txt"
output_dir = '/content/result'
overwrite_output_dir = False
num_train_epochs = 1
save_steps = 500

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(file_path, tokenizer, block_size = 128):
    # Will load and tokenize the data
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    # Helper Function 
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator

def train(
    train_file_path,
    model,
    tokenizer,
    output_dir,
    overwrite_output_dir,
    num_train_epochs,
    save_steps
    ):
  #TODO: Use the helper functions above to load the dataset and data collector.

  load_data_collator = load_dataset(train_file_path, tokenizer)

  # Don't mess with.
  tokenizer.save_pretrained(output_dir)
  model.save_pretrained(output_dir)

  # Don't mess with.
  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=8,
          num_train_epochs=num_train_epochs,
      )
  #TODO: Use the Trainer class with the necessary parameters to instaniate the trainer
  trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    )
  
  #TODO: Train and save the model using the train and save functions built into the Trainer class.
  trainer.train()
  trainer.save_model()

## Task 5: Creating a Tweet Generator
Now that we have our trained model, it's time to generate some Tweets. Since we should have saved our model and tokenizer to an output directory, I've already made some helper functions to load those in. We will focus on our *generate_text* function. The function will take as input some start text like "I am" or "My country is", etc., as well as a max_length parameter which tells the model how much text to generate. Let's Go!

In [ ]:
from transformers import pipeline

generate_txt = pipeline('text-generation',model='gpt2')
generate_txt('I can make')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 50 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


[{'generated_text': "I can make you stop watching me. How you doin' that? Ohhhhh! Are you doing it right? I bet the woman who was looking away from me got bored anyway. Maybe don't take it personally, if you wanna be on"}]